In [ ]:
from pathlib import Path

working = ["186"]

episodes = [
    "030",
    "051",
    "061",
    "069",
    "098",
    "118",
    "186",
    "240",
    "298",
    "398",
    "405",
    "430",
]

files_dir = Path("../files/text/rotl")

files = [
    file
    for file in sorted(files_dir.iterdir())
    if any(episode in str(file) for episode in working)
]

out_dir = Path("../files/chat")
out_dir.mkdir(exist_ok=True)

for file in files:
    text = open(file, "r").read()

In [2]:
from ollama import chat

stream = chat(
    model="command-r",
    messages=[{"role": "user", "content": "Why is the sky blue?"}],
    stream=True,
)

for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)

The sky appears blue to us during the day due to a phenomenon called Rayleigh scattering. When sunlight, which is composed of various colors of the visible spectrum, enters Earth's atmosphere, it interacts with gas molecules and tiny particles in the air. 

Blue light has a shorter wavelength compared to other colors like red or orange. This means that when sunlight encounters these small atmospheric particles (such as nitrogen and oxygen molecules), blue light is scattered more strongly than longer-wavelength colors. As a result, the blue light gets diffused in all directions by these molecules, making it reach our eyes from various angles. 

The other colors with longer wavelengths pass through the atmosphere relatively unscattered, reaching us directly from the sun and appearing less dominant in comparison to the scattered blue light. This is why we perceive the sky as being blue during the daytime when the sun is out.